In [15]:
import numpy as np
import time
import copy

In [2]:
nSamples = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [3]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# Generate

In [4]:
for x in nSamples:
    time = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 10)
        r = np.random.randint(time - 5 if time - 5 > 0 else 0 ,time + 5)
        d = np.random.randint(r+p, r+p+10)
        w = np.random.randint(0,10)
        time += p
        
        fileData.append([p,r,d,w])
    with open(f'data/instances_{x}.csv', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        for line in fileData[1:]:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

NameError: name 'n' is not defined

# Verify

In [5]:
x = 50

In [24]:
def calculateCost(data, taskQueue):
    calculatedWeight = 0
    timeCounter = 0
    for task in taskQueue:
        if timeCounter < data[task]['r']:
            timeCounter = data[task]['r']
        # add p time to current time
        timeCounter += data[task]['p']
        c = timeCounter
        data[task]['c'] = c

        # check correctness
        if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
            print("Error")

        # verify cost
        if data[task]["c"] > data[task]["d"]:
            calculatedWeight += data[task]["w"]
    #print(calculatedWeight)
    return calculatedWeight

In [10]:
motherFolder = "out" # "PTSZ-instancje"
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
#    print(index)
    for samples in nSamples:
        with open(f'{motherFolder}/{index}/out{index}_{samples}.txt', "r") as result:
            res = result.read().split('\n')
            weight = int(res[0])
            gen = (t for t in res[1].split(" ") if t.isdigit())
            taskQueue = [int(t) for t in gen]

        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])                
            for i,line in enumerate(d[1:]):
                if line == '':
                    continue
                p,r,d,w = line.split(' ')
                p,r,d,w = int(p), int(r), int(d), int(w)
                data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0 }
                    
        calculateCost(data, taskQueue)
        #if calculatedWeight != weight:
        #    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
        #else:
        #    print(f"Weight: {calculatedWeight} confirmed")
    #print()
    #print()
        
        

82
220
419
426
561
680
806
976
1174
1180
97
423
1014
1901
2783
4170
5853
8048
10243
11948
217
927
2201
3849
5803
8861
11955
16431
20297
24872
242
533
842
1148
1363
1653
1857
2187
2494
2792
0
38
774
1530
2075
2555
3086
3690
4208
4967
86
416
1076
2056
2915
4016
5977
7528
10473
12042
163
605
1263
2130
3455
4813
6397
7732
10208
13372
245
491
755
1039
1265
1539
1723
2054
2241
2468
88
145
287
331
546
566
556
869
734
1007
244
510
717
929
1192
1522
1713
2061
2187
2420
937
1653
2726
4038
4789
5451
6544
7496
8785
10634
0
0
0
0
0
0
0
0
0
0
1902
3887
5872
7894
9772
12017
13899
15638
17991
19600
189
341
1066
1326
1776
2056
3611
4056
4708
5191


In [11]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# TaskScheduler

In [56]:
totalCost = 0
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    for samples in nSamples:
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])                
            for i,line in enumerate(d[1:]):
                if line == '':
                    continue
                p,r,d,w = line.split(' ')
                p,r,d,w = int(p), int(r), int(d), int(w)
                data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
            dataCopy = copy.deepcopy(data)
        
                    
        calculatedWeight = 0
        timeCounter = 0
        outSchedule = []
        
        start = time.time()
        while len(data) > 0:
            readyToGo = {}
            for key, task in data.items():
                if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                    endTime = timeCounter+task['p']-task['d']
                    penaltyEndTask = ( endTime if endTime > 0 else 0 ) * task['w']
                    penaltyWeight = task['w']
                    penaltyPersistance = task['p']
                    penaltyDue = task['d'] - timeCounter
                    task['penalty'] = 10*penaltyEndTask + 30*penaltyWeight + 2*penaltyPersistance - 10*penaltyDue
                    readyToGo[key] = task
                
            if len(readyToGo) == 0: # currently there is no task to go
                #print("upsi")
                timeCounter += 1
                continue
            #print(f"readyToGo: {readyToGo}")

            readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
            selectedTask = readyToGoSorted.pop(0)
            #print(timeCounter)
            #print(f"selectedTask: {selectedTask}")

            data.pop(selectedTask[0])
            timeCounter += selectedTask[1]['p']
            outSchedule.append(selectedTask[0])    
        
        end = time.time()
        #print(end-start)
        
        cost = calculateCost(dataCopy, outSchedule)
        #print(cost)
        totalCost += cost
        
        with open(f'out/{index}/out{index}_{samples}.txt', "w") as f:
            f.write(f"{cost}\n")         
                #f.write(f"{' '.join([str(x) for x in line])}\n")
            f.write(f"{' '.join([str(x) for x in outSchedule])}\n")
        
print(totalCost)

497405


In [28]:
497400

501021

In [57]:
from scipy.optimize import minimize

In [81]:
x0 = (1, 1, 1, 1)

In [93]:
def costFunction(x):
    a,b,c,d = x
    totalCost = 0
    data = {}
    indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
    for index  in indicies:
        for samples in nSamples:
            with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
                d = instance.read().split('\n')
                n = int(d[0])                
                for i,line in enumerate(d[1:]):
                    if line == '':
                        continue
                    p,r,d,w = line.split(' ')
                    p,r,d,w = int(p), int(r), int(d), int(w)
                    data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
                dataCopy = copy.deepcopy(data)


            calculatedWeight = 0
            timeCounter = 0
            outSchedule = []

            start = time.time()
            while len(data) > 0:
                readyToGo = {}
                for key, task in data.items():
                    if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                        endTime = timeCounter+task['p']-task['d']
                        penaltyEndTask = ( endTime if endTime > 0 else 0 ) * task['w']
                        penaltyWeight = task['w']
                        penaltyPersistance = task['p']
                        penaltyDue = task['d'] - timeCounter
                        task['penalty'] = a*penaltyEndTask + b*penaltyWeight + c*penaltyPersistance - d*penaltyDue
                        readyToGo[key] = task

                if len(readyToGo) == 0: # currently there is no task to go
                    #print("upsi")
                    timeCounter += 1
                    continue
                #print(f"readyToGo: {readyToGo}")

                readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
                selectedTask = readyToGoSorted.pop(0)
                #print(timeCounter)
                #print(f"selectedTask: {selectedTask}")

                data.pop(selectedTask[0])
                timeCounter += selectedTask[1]['p']
                outSchedule.append(selectedTask[0])    

            end = time.time()
            print(int((end-start)*1000) if int((end-start)*1000) != 0 else 1)

            cost = calculateCost(dataCopy, outSchedule)
            #print(cost)
            totalCost += cost
    #print(f"[{totalCost},{a},{b},{c},{d}],")
    return totalCost

In [83]:
options = {"maxiter":40, "disp":True}

In [84]:
res = minimize(costFunction, x0, method='Powell', options=options, tol=1e-2)

499007
1.0 1.0 1.0 1183
499007
1.0 1.0 1.0 1183
499045
2.0 1.0 1.0 1183
497286
-0.618034 1.0 1.0 1183
494946
-3.2360680251559995 1.0 1.0 1183
488005
-13.073682999334991 1.0 1.0 1183
481376
-28.991278506465722 1.0 1.0 1183
477630
-39.5830756433936 1.0 1.0 1183
473386
-56.72096353204556 1.0 1.0 1183
467852
-80.53361600485479 1.0 1.0 1183
457390
-119.0632973360442 1.0 1.0 1183
443693
-181.40563173907393 1.0 1.0 1183
423644
-364.0535394470477 1.0 1.0 1183
410253
-659.5840641474113 1.0 1.0 1183
409480
-679.893230953948 1.0 1.0 1183
408255
-712.7541533585958 1.0 1.0 1183
396205
-1961.526600261653 1.0 1.0 1183
392702
-3982.0828776139942 1.0 1.0 1183
393653
-3329.8213941069635 1.0 1.0 1183
390669
-7251.411633283513 1.0 1.0 1183
390737
-7096.03416767051 1.0 1.0 1183
388315
-12541.296717134486 1.0 1.0 1183
387035
-20665.559772087272 1.0 1.0 1183
386617
-33810.89361994475 1.0 1.0 1183
386714
-29927.350203028738 1.0 1.0 1183
386395
-55080.49072712897 1.0 1.0 1183
386412
-52853.65178405192 1.0 1.0 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = [1,2,3]

In [ ]:
plt.plot(x)
plt.title("Minimalizacja funkcji celu")
plt.xaxis("iteracja")
plt.yaxis("Koszt")

In [94]:
costFunction(x)

1
7
13
25
17
19
26
33
38
61
1
4
10
17
29
42
64
85
99
140
1
4
11
19
30
45
63
152
111
133
1
3
15
16
25
32
48
61
82
88
1
1
6
13
30
36
48
64
94
108
1
3
6
11
19
27
38
52
62
79
1
3
8
22
29
38
56
71
105
124
1
4
10
17
26
39
53
69
105
107
1
1
1
2
2
4
7
6
9
10
1
3
6
10
16
26
33
54
57
68
1
3
7
15
22
45
45
56
75
108
1
1
1
1
2
4
5
7
8
11
1
4
11
21
32
47
74
87
110
146
1
3
5
8
14
28
34
39
54
84


385570

In [85]:
x = (-140036.8041378972, 2.5486165590751364, -0.1998934445512417, 1183)